# POC 激活验证
### 1. 获得异构图

In [ ]:
import dgl
import torch
from model_lib.hetero_struct_backdoor_dataset import HeteroStrucBackdoorDataset
from utils_gnn import all_u_to_v

def cnn2graph(model, model_info):
    # convert cnn model to a dgl graph
    # model: model weight data
    # model_info: model struct info
    
    graph_data = {
        # (src_type, relation, dst_type): ([src_nodes], [dst_nodes]) type in tensor
        ('conv', 'channels_to', 'conv'): None,
        ('conv', 'concat_to', 'fc'): None,
        ('fc', 'connect_to', 'fc'): None
    }
    layers = []
    all_node_feats = []
    all_edges = []
    cnt = 0
    layer_types = [] 
    layer_idx = []
    with torch.no_grad():
        for i in range(len(model_info)):
            cur_layer_info = model_info[i]
            cur_layer_node = []
            
            if 'conv' in cur_layer_info['name']:
                # construct cur layer nodes
                for weight, bias in zip(model.get_submodule(cur_layer_info['name']).weight, 
                                        model.get_submodule(cur_layer_info['name']).bias):
                    cur_layer_node.append(cnt)
                    cnt += 1
                    # featue resize?
                    w = weight[0] + bias
                    # all_node_feats.append(padding(w, 512, 513))
                    layer_types.append('conv')
                    layer_idx.append(i)
            else:
                # construct dense layer node
                cur_layer_node.append(cnt)
                cnt += 1
                # feature resize?
                w = model.get_submodule(cur_layer_info['name']).weight.t() + model.get_submodule(cur_layer_info['name']).bias
                all_node_feats.append(padding(w, 512, 513))
                layer_types.append('fc')
                layer_idx.append(i)
            layers.append(cur_layer_node)
            
    # get all edges
    conv_2_conv = []
    conv_2_fc = []
    fc_2_fc =[]
    for idx in range(len(layers)):
        if idx < len(layers) - 1:
            cur_layer_type = layer_types[idx]
            next_layer_type = layer_types[idx+1]
            if cur_layer_type == 'conv' and next_layer_type == 'conv':
                conv_2_conv += all_u_to_v(layers[idx], layers[idx+1])
            elif cur_layer_type == 'conv' and next_layer_type == 'fc':
                conv_2_fc += all_u_to_v(layers[idx], layers[idx+1])
            else:
                fc_2_fc += all_u_to_v(layers[idx], layers[idx+1])
    
    # convert to tensor
    graph_data[('conv', 'channels_to', 'conv')] = torch.tensor(conv_2_conv).t()
    graph_data[('conv', 'concat_to', 'fc')] = torch.tensor(conv_2_fc).t()
    graph_data[('fc', 'connect_to', 'fc')] = torch.tensor(fc_2_fc).t()

    # get hetero graph
    g = dgl.heterograph(graph_data).to('cuda')
    g.ndata['x'] = torch.stack(all_node_feats)
    return g
